In [ ]:
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import datetime as dt
import os
from pyproj import Proj, transform

root=os.path.abspath(os.curdir)
oldpath=os.path.join(root,'OlD Data')
newpath=os.path.join(root,'latest')

inProj = Proj('epsg:3857')
outProj = Proj('epsg:4326')

today=date.today()
def BF(df,day):
    return df+str(day)+'.csv'


normalized_district = {
    'SHATIN':'SHA TIN',
    'ISLAND':'ISLANDS',
    'CENTRAL &WESTERN':'CENTRAL & WESTERN'
}

def normalize_district(district):
    """Convert fuck_up_district to normalized_district.
    """
    if district not in normalized_district :
        return district

    return normalized_district[district]

url='https://services8.arcgis.com/PXQv9PaDJHzt8rp0/arcgis/rest/services/StayBuildingWithHistory_0227_View/FeatureServer/0/query?f=json&where=Status_Chi%20%3D%20N%27%E7%8F%BE%E6%9C%89%27&returnGeometry=true&spatialRel=esriSpatialRelIntersects&geometryType=esriGeometryEnvelope&inSR=102100&outFields=*&outSR=102100&resultType=tile'

url1='https://services8.arcgis.com/PXQv9PaDJHzt8rp0/arcgis/rest/services/StayBuildingWithHistory_0227_View/FeatureServer/0/query?f=json&where=Status_Chi%20%3D%20N%27%E9%81%8E%E5%8E%BB%27&returnGeometry=true&spatialRel=esriSpatialRelIntersects&geometryType=esriGeometryEnvelope&inSR=102100&outFields=*&outSR=102100&resultType=tile'





s=requests.get(url)

df=pd.DataFrame(columns=json.loads(s.content)['features'][0]['attributes'].keys())
for x in json.loads(s.content)['features']:
    df=df.append(x['attributes'],ignore_index=True)
    
df1=pd.DataFrame(columns=json.loads(s.content)['features'][0]['geometry'].keys())
for x in json.loads(s.content)['features']:
    df1=df1.append(x['geometry'],ignore_index=True)    
    
df = pd.concat([df, df1], axis=1)  


s=requests.get(url1)
df2=pd.DataFrame(columns=json.loads(s.content)['features'][0]['attributes'].keys())
for x in json.loads(s.content)['features']:
    df2=df2.append(x['attributes'],ignore_index=True)

df3=pd.DataFrame(columns=json.loads(s.content)['features'][0]['geometry'].keys())
for x in json.loads(s.content)['features']:
    df3=df3.append(x['geometry'],ignore_index=True)   
    
df2 = pd.concat([df2, df3], axis=1)  
    
df=df2.append(df)

df=df.drop(columns=['StayedBuildingID','OBJECTID','地區','大廈名單','Creator','CreationDate','EditDate','Editor','Status_Chi','相關確診個案','GlobalID','EndDate','FirstReportedDate','最後有個案在出現病徵期間逗留的日期','SpecialCase'])      

building=df

building['DateoftheLastCase'] = building['DateoftheLastCase'].apply(lambda x: "Unknown" if x == None else(x))
building['DateoftheLastCase'] = building['DateoftheLastCase'].apply(lambda x: str(x) if x != "Unknown" else(x))
building['DateoftheLastCase'] = building['DateoftheLastCase'].apply(lambda x : x[:-3] if x != "Unknown" else(x))
building['DateoftheLastCase'] = building['DateoftheLastCase'].apply(lambda x: int(x) if x != "Unknown" else(x))

building_1=building

building_1['DateoftheLastCase']=building_1['DateoftheLastCase'].apply(lambda x: datetime.fromtimestamp(x) if x != "Unknown" else(x))
building_1['DateoftheLastCase']=building_1['DateoftheLastCase'].apply(lambda x: dt.datetime.strftime(x,'%Y-%m-%d') if x != "Unknown" else(x) )

building_1['District']=building_1['District'].str.upper()
building_1['District']=building_1['District'].apply(lambda x: x.strip())
building_1['District']=building_1['District'].apply(lambda x: normalize_district(x))


building_1=building_1.rename(columns={"BuildingName": "Building name", "Related_confirmed_cases": "Case Number","DateoftheLastCase": "Date"})
building_1['Address'] = building_1['Building name']+','+building_1['District']+','+'Hong Kong'

building_1['x'],building_1['y'] = transform(inProj, outProj, (building_1['x'].to_numpy()), (building_1['y'].to_numpy()))

building_1["Case Number"]=building_1["Case Number"].str.split(",")
building_1 = building_1.explode("Case Number").reset_index(drop=True)

building_1['count'] = pd.DataFrame([1] * building_1.shape[0])

building_1['Building name']=building_1['Building name'].apply(lambda x: x.strip())
building_1['residential']=building_1['Building name'].apply(lambda x: x.find(('non-residential')))
building_1['residential']=building_1['residential'].apply(lambda x: "TRUE" if x == -1 else('FALSE'))

building_1['Address'] = building_1['Address'].apply(lambda x: 'Unknow Address' if x == ',,Hong Kong' else(x))

building_1 = building_1.drop_duplicates()

building_1.to_csv(os.path.join(newpath,'MAPbuildingEDIT.csv'),index=False)

try:
#shutil.move(os.path.join(newpath,BF('buildingdegree3above',yesterday)), oldpath)
#    shutil.move(os.path.join(newpath,BF('buildingCase',yesterday)), oldpath)
   # shutil.move(os.path.join(newpath,BF('flightdegree3above',yesterday)), oldpath)
    shutil.move(os.path.join(newpath,'MAPbuildingDONTEDIT.csv'), oldpath)    
   # shutil.move(os.path.join(newpath,BF('flight',yesterday)), oldpath)

    
    
except:
    print('No such file')
    
